# Pandas:
## data structures, IO, aggregation, melt/pivot, group operations, merge/join

this basically covers the concepts you need to master with Pandas

Author: [Alexandre Gramfort](http://alexandre.gramfort.net)

(based on Wes McKinney's book "Python for Data Analysis" O'REILLY,
and the pandas materials from Joris Van den Bossche and Stijn Van Hoey.)

Licensed under [CC BY 4.0 Creative Commons](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
import pandas as pd  # THE standard way of importing pandas

Some imports

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-colorblind')

# Data structures

Pandas provides two fundamental data objects, for 1D (``Series``) and 2D data (``DataFrame``).

## Series

A Series is a basic holder for **one-dimensional labeled data**. It can be created much as a NumPy array is created:

In [ ]:
s = pd.Series([0.1, 0.2, 0.3, 0.4])
s

In [ ]:
type(s)

### Attributes of a Series: `index` and `values`

The series has a built-in concept of an **index**, which by default is the numbers *0* through *N - 1*

In [ ]:
s.index

You can access the underlying numpy array representation with the `.values` attribute:

In [ ]:
s.values

In [ ]:
type(s.values)

We can access series values via the index, just like for NumPy arrays:

In [ ]:
s[0]

Unlike the NumPy array, though, this index can be something other than integers:

In [ ]:
s2 = pd.Series(np.arange(4), index=['a', 'b', 'c', 'd'])
s2

In [ ]:
s2['c']

### Pandas Series versus dictionaries

In this way, a ``Series`` object can be thought of as similar to an ordered dictionary mapping one typed value to another typed value.

In fact, it's possible to construct a series directly from a Python dictionary:

In [ ]:
pop_dict = {'Germany': 81.3, 
            'Belgium': 11.3, 
            'France': 64.3, 
            'United Kingdom': 64.9, 
            'Netherlands': 16.9}
population = pd.Series(pop_dict)
population

We can index the populations like a dict as expected:

In [ ]:
population['France']

but with the power of numpy arrays:

In [ ]:
population * 1000

## DataFrames: Multi-dimensional Data

A DataFrame is a **tablular data structure** (multi-dimensional object to hold labeled data) comprised of rows and columns, akin to a spreadsheet, database table, or R's data.frame object. You can think of it as multiple Series object which share the same index.

<img src="img/schema-dataframe.svg" width=50%>

One of the most common ways of creating a dataframe is from a dictionary of arrays or lists.

Note that in the IPython notebook, the dataframe will display in a rich HTML view:

In [ ]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

In [ ]:
type(countries['area'])

### Attributes of the DataFrame

A DataFrame has besides a `index` attribute, also a `columns` attribute:

In [ ]:
countries.index

In [ ]:
countries.columns

To check the data types of the different columns:

In [ ]:
countries.dtypes

An overview of that information can be given with the `info()` method:

In [ ]:
countries.info()

Also a DataFrame has a `values` attribute, but attention: when you have heterogeneous data, all values will be upcasted:

In [ ]:
countries.values

To access a Series representing a column in the data, use typical indexing syntax:

In [ ]:
countries['area']

### Changing the DataFrame index

If we don't like what the index looks like, we can reset it and set one of our columns:

In [ ]:
countries = countries.set_index('country')
countries

Reversing this operation, is `reset_index`:

In [ ]:
countries.reset_index('country')

## Some useful methods on these data structures

Exploration of the Series and DataFrame is essential (check out what you're dealing with). 

In [ ]:
countries.head(4) # Top rows

In [ ]:
countries.tail(4) # Bottom rows

One useful method to use is the ``describe`` method, which computes summary statistics for each column:

In [ ]:
countries.describe()

**Sort**ing your data **by** a specific column is another important first-check:

In [ ]:
countries.sort_values(by='population')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Check the help of the `sort_values` function and find out how to sort from the largest values to the lowest values</li>
    </ul>
</div>

The **`plot`** method can be used to quickly visualize the data in different ways:

In [ ]:
countries.plot()

However, for this dataset, it does not say that much:

In [ ]:
countries

In [ ]:
countries['population'].plot(kind='barh')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>You can play with the `kind` keyword of the `plot` function in the figure above: 'line', 'bar', 'hist', 'density', 'area', 'pie', 'scatter', 'hexbin'</li>
    </ul>
</div>

# Importing and exporting data

A wide range of input/output formats are natively supported by pandas:

* CSV, text
* SQL database
* Excel
* HDF5
* json
* html
* pickle
* ...

In [ ]:
countries.to_dict()

##  Basic operations on Series and DataFrames

As you play around with DataFrames, you'll notice that many operations which work on NumPy arrays will also work on dataframes.

In [ ]:
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

countries = pd.DataFrame({'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
                          'population': [11.3, 64.3, 81.3, 16.9, 64.9],
                          'area': [30510, 671308, 357050, 41526, 244820],
                          'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']})

In [ ]:
countries.head()

# The 'new' concepts

## Elementwise-operations 

Just like with numpy arrays, many operations are element-wise:

In [ ]:
population

In [ ]:
population / 100

In [ ]:
density = countries['population'] / countries['area']

In [ ]:
density

In [ ]:
np.log(countries['population'])

which can be added as a new column, as follows:

In [ ]:
countries["log_population"] = np.log(countries['population'])

In [ ]:
countries.columns

In [ ]:
countries.head(3)

In [ ]:
countries['population'] > 40

<div class="alert alert-info">

<b>REMEMBER</b>:

<ul>
  <li>When you have an operation which does NOT work element-wise or you have no idea how to do it directly in Pandas, use the **apply()** function</li>
  <li>A typical use case is with a custom written or a **lambda** function</li>
</ul>
</div>

In [ ]:
countries["population"].apply(np.log) # but this works as well element-wise...

In [ ]:
# in case you forgot the functionality: countries["capital"].str.len()
countries["capital"].apply(lambda x: len(x))

In [ ]:
def population_annotater(population):
    """annotate as large or small"""
    if population > 50:
        return 'large'
    else:
        return 'small'

In [ ]:
countries["population"].apply(population_annotater) # a custom user function

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the population numbers relative to France</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the population density for each country and add this as a new column to the dataframe.</li>
</ul>
</div>

<div class="alert alert-danger">

**WARNING**: **Alignment!** (unlike numpy)

 <ul>
  <li>Pay attention to **alignment**: operations between series will align on the index:  </li>
</ul> 

</div>

In [ ]:
s1 = population[['Belgium', 'France']]
s2 = population[['France', 'Germany']]

In [ ]:
s1

In [ ]:
s2

In [ ]:
s1 + s2

## Aggregations (reductions)

Pandas provides a large set of **summary** functions that operate on different kinds of pandas objects (DataFrames, Series, Index) and produce single value. When applied to a DataFrame, the result is returned as a pandas Series (one value for each column). 

The average population number:

In [ ]:
population.mean()

The minimum area:

In [ ]:
countries['area'].min()

For dataframes, often only the numeric columns are included in the result:

In [ ]:
countries.median()

## Pandas: Indexing and selecting data

In [ ]:
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

Setting the index to the country names:

In [ ]:
countries = countries.set_index('country')
countries

# Selecting data

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. <br><br> We now have to distuinguish between:

 <ul>
  <li>selection by **label**</li>
  <li>selection by **position**</li>
</ul>
</div>

## `data[]` provides some convenience shortcuts 

For a DataFrame, basic indexing selects the columns (cfr. the dictionaries of pure python)

Selecting a **single column**:

In [ ]:
countries['area'] # single []

or multiple **columns**:

In [ ]:
countries[['area', 'population']] # double [[]]

But, slicing accesses the **rows**:

In [ ]:
countries['France':'Netherlands']

<div class="alert alert-danger">
<b>NOTE</b>:

 <ul>
  <li>Unlike slicing in numpy, the end label is **included**!</li>
</ul>
</div>

<div class="alert alert-info">
<b>REMEMBER</b>: <br><br>

So as a summary, `[]` provides the following convenience shortcuts:

<ul>
  <li>**Series**: selecting a **label**: `s[label]`</li>
  <li>**DataFrame**: selecting a single or multiple **columns**: `df['col']` or `df[['col1', 'col2']]`</li>
  <li>**DataFrame**: slicing the **rows**: `df['row_label1':'row_label2']` or `df[mask]`</li>
</ul>
</div>

## Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

Selecting a single element:

In [ ]:
countries.loc['Germany', 'area']

But the row or column indexer can also be a list, slice, boolean array (see next section), ..

In [ ]:
countries.loc['France':'Germany', ['area', 'population']]

---
Selecting by position with `iloc` works similar as **indexing numpy arrays**:

In [ ]:
countries.iloc[0:2,1:3]

The different indexing methods can also be used to assign data:

In [ ]:
countries2 = countries.copy()
countries2.loc['Belgium':'Germany', 'population'] = 10

In [ ]:
countries2

<div class="alert alert-info">
<b>REMEMBER</b>: <br><br>

Advanced indexing with **loc** and **ïloc**

 <ul>
  <li>**loc**: select by label: `df.loc[row_indexer, column_indexer]`</li>
  <li>**iloc**: select by position: `df.iloc[row_indexer, column_indexer]`</li>
</ul>
</div>

## Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with 'boolean indexing' (like a where clause in SQL) and comparable to numpy. 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
countries['area'] > 100000

In [ ]:
countries[countries['area'] > 100000]

<div class="alert alert-success">
<b>EXERCISE</b>:

<ul>
    <li>Add density as column to the DataFrame</li>
</ul>
</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

 <ul>
  <li>Select the capital and the population column of those countries where the density is larger than 300</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Add a column 'density_ratio' with the ratio of the density to the mean density</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Change the capital of the UK to Cambridge</li>
</ul>
</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

 <ul>
  <li>Select all countries whose population density is between 100 and 300 people/km²</li>
</ul>
</div>

# Some other essential methods: `isin` and `string` methods

The `isin` method of Series is very useful to select rows that may contain certain values:

In [ ]:
s = countries['capital']

In [ ]:
s.isin?

In [ ]:
s.isin(['Berlin', 'London'])

This can then be used to filter the dataframe with boolean indexing:

In [ ]:
countries[countries['capital'].isin(['Berlin', 'London'])]

Let's say we want to select all data for which the capital starts with a 'B'. In Python, when having a string, we could use the `startswith` method:

In [ ]:
'Berlin'.startswith('B')

In pandas, these are available on a Series through the `str` namespace:

In [ ]:
countries['capital'].str.startswith('B')

For an overview of all string methods, see: http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Select all countries that have capital names with more than 7 characters</li>
</ul>
    
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Select all countries that have capital names that contain the character sequence 'am'</li>
</ul>
</div>

# Pitfall: chained indexing (and the 'SettingWithCopyWarning')

In [ ]:
countries.loc['Belgium', 'capital'] = 'Paris'

In [ ]:
countries

In [ ]:
countries['capital']['Belgium'] = 'Lyon' 

In [ ]:
countries

In [ ]:
countries[countries['capital'] == 'Lyon']['capital'] = 'Brussels' 

In [ ]:
countries

<div class="alert alert-info">

<b>REMEMBER!</b><br><br>

What to do when encountering the *value is trying to be set on a copy of a slice from a DataFrame* error?

 <ul>
  <li>Use `loc` instead of chained indexing **if possible**!</li>
  <li>Or `copy` explicitly if you don't want to change the original data.</li>
</ul>

</div>

## Pandas: Working with time series data

# Introduction: `datetime` module

Standard Python contains the `datetime` module to handle with date and time data:

In [ ]:
import datetime

In [ ]:
dt = datetime.datetime(year=2016, month=12, day=19, hour=13, minute=30)
dt

In [ ]:
print(dt) # .day,...

In [ ]:
print(dt.day, dt.year, dt.month)

In [ ]:
print(dt.strftime("%d %B %Y"))

# Dates and times in pandas

## The ``Timestamp`` object

Pandas has its own date and time objects, which are compatible with the standard `datetime` objects, but provide some more functionality to work with.  

The `Timestamp` object can also be constructed from a string:

In [ ]:
ts = pd.Timestamp('2016-12-19')
ts

Like with `datetime.datetime` objects, there are several useful attributes available on the `Timestamp`. For example, we can get the month:

In [ ]:
ts.month

In [ ]:
pd.Timedelta('5 days')

In [ ]:
ts + pd.Timedelta('5 days')

### Parsing datetime strings 

![](http://imgs.xkcd.com/comics/iso_8601.png)


Unfortunately, when working with real world data, you encounter many different `datetime` formats. Most of the time when you have to deal with them, they come in text format, e.g. from a `CSV` file. To work with those data in Pandas, we first have to *parse* the strings to actual `Timestamp` objects.

<div class="alert alert-info">
<b>REMEMBER</b>: <br><br>

from string formatted dates to Timestamp objects: `to_datetime` function
</div>



In [ ]:
pd.to_datetime("2016-12-09")

In [ ]:
pd.to_datetime("09/12/2016")

In [ ]:
pd.to_datetime("09/12/2016", dayfirst=True)

In [ ]:
pd.to_datetime("09/12/2016", format="%d/%m/%Y")

A detailed overview of how to specify the `format` string, see the table in the python documentation: https://docs.python.org/3.5/library/datetime.html#strftime-and-strptime-behavior

## `Timestamp` data in a Series or DataFrame column

In [ ]:
s = pd.Series(['2016-12-09 10:00:00', '2016-12-09, 11:00:00',
               '2016-12-09 12:00:00'])

In [ ]:
s

The `to_datetime` function can also be used to convert a full series of strings:

In [ ]:
ts = pd.to_datetime(s)

In [ ]:
ts

Notice the data type of this series: the `datetime64[ns]` dtype. This indicates that we have a series of actual datetime values.

The same attributes as on single `Timestamp`s are also available on a Series with datetime data, using the **`.dt`** accessor:

In [ ]:
ts.dt.hour

In [ ]:
ts.dt.weekday

To quickly construct some regular time series data, the [``pd.date_range``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.date_range.html) function comes in handy:

In [ ]:
pd.Series(pd.date_range(start="2016-01-01", periods=10, freq='3H'))

In [ ]:
pd.Series(pd.date_range(start="2016-01-01", periods=10, freq='1D'))

# Time series data: `Timestamp` in the index

## River discharge example data

For the following demonstration of the time series functionality, we use a sample of discharge data of the Maarkebeek (Flanders) with 3 hour averaged values, derived from the [Waterinfo website](https://www.waterinfo.be/).

In [ ]:
data = pd.read_csv("data/vmm_flowdata.csv")

In [ ]:
data.info()

In [ ]:
data.head()

We already know how to parse a date column with Pandas:

In [ ]:
data['Time'] = pd.to_datetime(data['Time'])

In [ ]:
data.info()

With `set_index('datetime')`, we set the column with datetime values as the index, which can be done by both `Series` and `DataFrame`.

In [ ]:
data = data.set_index("Time")

In [ ]:
data

The steps above are provided as built-in functionality of `read_csv`:

In [ ]:
data = pd.read_csv("data/vmm_flowdata.csv", index_col=0, parse_dates=True)

In [ ]:
data.head()

<div class="alert alert-info">
<b>REMEMBER</b>: <br><br>

`pd.read_csv` provides a lot of built-in functionality to support this kind of transactions when reading in a file! Check the help of the read_csv function...
</div>

## The DatetimeIndex

When we ensure the DataFrame has a `DatetimeIndex`, time-series related functionality becomes available:

In [ ]:
data.index

Similar to a Series with datetime data, there are some attributes of the timestamp values available:

In [ ]:
data.index.day

In [ ]:
data.index.dayofyear

In [ ]:
data.index.year

The `plot` method will also adapt it's labels (when you zoom in, you can see the different levels of detail of the datetime labels):


In [ ]:
data.plot()

We have to much data to sensibly plot on one figure. Let's see how we can easily select part of the data or aggregate the data to other time resolutions in the next sections.

## Selecting data from a time series

We can use label based indexing on a timeseries as expected:

In [ ]:
data[pd.Timestamp("2012-01-01 09:00"):pd.Timestamp("2012-01-01 19:00")]

But, for convenience, indexing a time series also works with strings:

In [ ]:
data["2012-01-01 09:00":"2012-01-01 19:00"]

A nice feature is **"partial string" indexing**, where we can do implicit slicing by providing a partial datetime string.

E.g. all data of 2013:

In [ ]:
data['2013']

Normally you would expect this to access a column named '2013', but as for a DatetimeIndex, pandas also tries to interprete it as a datetime slice.

Or all data of January up to March 2012:

In [ ]:
data['2012-01':'2012-03']

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>select all data starting from 2012</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>select all data in January for all different years</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>select all data in January, February and March for all different years</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>select all 'daytime' data (between 8h and 20h) for all days</li>
</ul>
</div>

## The power of pandas: `resample`

A very powerfull method is **`resample`: converting the frequency of the time series** (e.g. from hourly to daily data).

The time series has a frequency of 1 hour. I want to change this to daily:

In [ ]:
data.resample('D').mean().head()

<div class="alert alert-danger">

<b>NOTE</b>:

 <ul>
  <li>with older versions of pandas, <code>data.resample('D').mean()</code> was expressed as <code>data.resample('D', how='mean')</code>.</li>
</ul>
</div>

Other mathematical methods can also be specified:

In [ ]:
data.resample('D').max().head()

<div class="alert alert-info">
<b>REMEMBER</b>: <br><br>
    The string to specify the new time frequency: http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases <br><br>
    These strings can also be combined with numbers, eg `'10D'`...
</div>



In [ ]:
data.resample('A').mean().plot()

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>plot the monthly standard deviation of the columns</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:
 <ul>
  <li>plot the monthly mean and median values for the years 2011-2012 for 'L06_347'<br><br></li>
 </ul>
     <b>Note** </b> <br>You can create a new figure with  `fig, ax = plt.subplots()`
 and add each of the plots to the created `ax` object (see documentation of pandas plot function)
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>plot the monthly mininum and maximum daily average value of the 'LS06_348' column</li>
</ul>
</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

 <ul>
  <li>make a bar plot of the mean of the stations in year of 2013 (Remark: create a `fig, ax = plt.subplots()` object and add the plot to the created ax</li>
</ul>

</div>

## Combining datasets Part I - concat

Combining data is essential functionality in a data analysis workflow. 

Data is distributed in multiple files, different information needs to be merged, new data is calculated, .. and needs to be added together. Pandas provides various facilities for easily combining together Series and DataFrame objects

In [ ]:
# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

# Adding columns

As we already have seen before, adding a single column is very easy:

In [ ]:
pop_density = countries['population'] * 1e6 / countries['area']

In [ ]:
pop_density

In [ ]:
countries['pop_density'] = pop_density

In [ ]:
countries

Adding multiple columns at once is also possible. For example, the following method gives us a DataFrame of two columns:

In [ ]:
countries["country"].str.split(" ", expand=True)

We can add both at once to the dataframe:

In [ ]:
countries[['first', 'last']] = countries["country"].str.split(" ", expand=True)

In [ ]:
countries

Delete the new columns to revert the change

In [ ]:
countries.drop(['first', 'last'], axis=1, inplace=True)
countries

# Concatenating data

The ``pd.concat`` function does all of the heavy lifting of combining data in different ways.

``pd.concat`` takes a list or dict of Series/DataFrame objects and concatenates them in a certain direction (`axis`) with some configurable handling of “what to do with the other axes”.


## Combining rows - ``pd.concat``

![](img/schema-concat0.svg)

Assume we have some similar data as in `countries`, but for a set of different countries:

In [ ]:
data = {'country': ['Nigeria', 'Rwanda', 'Egypt', 'Morocco', ],
        'population': [182.2, 11.3, 94.3, 34.4],
        'area': [923768, 26338 , 1010408, 710850],
        'capital': ['Abuja', 'Kigali', 'Cairo', 'Rabat']}
countries_africa = pd.DataFrame(data)
countries_africa 

We now want to combine the rows of both datasets:

In [ ]:
pd.concat([countries, countries_africa])

If we don't want the index to be preserved:

In [ ]:
pd.concat([countries, countries_africa], ignore_index=True)

When the two dataframes don't have the same set of columns, by default missing values get introduced:

In [ ]:
pd.concat([countries, countries_africa[['country', 'capital']]],
          ignore_index=True)

We can also pass a dictionary of objects instead of a list of objects. Now the keys of the dictionary are preserved as an additional index level:

In [ ]:
pd.concat({'europe': countries, 'africa': countries_africa})

## Combining columns  - ``pd.concat`` with ``axis=1``

![](img/schema-concat1.svg)

Assume we have another DataFrame for the same countries, but with some additional statistics:

In [ ]:
data = {'country': ['Belgium', 'France', 'Netherlands'],
        'GDP': [496477, 2650823, 820726],
        'area': [8.0, 9.9, 5.7]}
country_economics = pd.DataFrame(data).set_index('country')
country_economics

In [ ]:
pd.concat([countries, country_economics], axis=1)

`pd.concat` matches the different objects based on the index:

In [ ]:
countries2 = countries.set_index('country')

In [ ]:
countries2

In [ ]:
pd.concat([countries2, country_economics], axis=1)

# Joining data with `pd.merge`

Using `pd.concat` above, we combined datasets that had the same columns or the same index values. But, another typical case if where you want to add information of second dataframe to a first one based on one of the columns. That can be done with [`pd.merge`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).

Let's look again at the titanic passenger data, but taking a small subset of it to make the example easier to grasp:

In [ ]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom', 'Australia'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9, 24.4],
        'area': [30510, 671308, 357050, 41526, 244820, 7692024],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London', 'Canberra'],
        'hemisphere': ['North', 'North', 'North', 'North', 'North', 'South']}
countries = pd.DataFrame(data)
countries

In [ ]:
data = {'city': ['Paris', 'Lyon', 'Lille', 'Berlin', 'Munich', 'Sydney'],
        'country': ['France', 'France', 'France', 'Germany', 'Germany', 'Australia'],
        }
cities = pd.DataFrame(data)
cities

Let's say we want to know in what hemisphere is each city

In [ ]:
pd.merge(cities, countries, on='country', how='left')

In [ ]:
pd.merge(cities, countries[['country', 'hemisphere']],
         on='country', how='left')

In this case we use `how='left` (a "left join") because we wanted to keep the original rows of `cities` and only add matching values from `countries` to it. Other options are 'inner', 'outer' and 'right' (see the [docs](http://pandas.pydata.org/pandas-docs/stable/merging.html#brief-primer-on-merge-methods-relational-algebra) for more on this).

## "Group by" operations

### Some 'theory': the groupby operation (split-apply-combine)

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

### Recap: aggregating functions

When analyzing data, you often calculate summary statistics (aggregations like the mean, max, ...). As we have seen before, we can easily calculate such a statistic for a Series or column using one of the many available methods. For example:

In [ ]:
df['data'].sum()

However, in many cases your data has certain groups in it, and in that case, you may want to calculate this statistic for each of the groups.

For example, in the above dataframe `df`, there is a column 'key' which has three possible values: 'A', 'B' and 'C'. When we want to calculate the sum for each of those groups, we could do the following:

In [ ]:
print('A', df[df['key'] == "A"]['data'].sum())
print('B', df[df['key'] == "B"]['data'].sum())
print('C', df[df['key'] == "C"]['data'].sum())

This becomes very verbose when having multiple groups. You could make the above a bit easier by looping over the different values, but still, it is not very convenient to work with.

What we did above, applying a function on different groups, is a "groupby operation", and pandas provides some convenient functionality for this.

### Groupby: applying functions per group

The "group by" concept: we want to **apply the same function on subsets of your dataframe, based on some key to split the dataframe in subsets**

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

<img src="img/splitApplyCombine.png">

Similar to SQL `GROUP BY`

Instead of doing the manual filtering as above


    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...

pandas provides the `groupby` method to do exactly this:

In [ ]:
df.groupby('key').sum()

In [ ]:
df.groupby('key').aggregate(np.sum)  # 'sum'

And many more methods are available. 

In [ ]:
df.groupby('key')['data'].sum()

# Some more theory

## Specifying the grouper

In the previous example and exercises, we always grouped by a single column by passing its name. But, a column name is not the only value you can pass as the grouper in `df.groupby(grouper)`. Other possibilities for `grouper` are:

- a list of strings (to group by multiple columns)
- a Series (similar to a string indicating a column in df) or array
- function (to be applied on the index)
- dict : groups by values
- levels=[], names of levels in a MultiIndex

In [ ]:
df.groupby(df['data'] < 18).mean()

## The size of groups - value counts

Oftentimes you want to know how many elements there are in a certain group (or in other words: the number of occurences of the different values from a column).

To get the size of the groups, we can use `size`:

In [ ]:
df.groupby('key').size()

Another way to obtain such counts, is to use the Series `value_counts` method:

In [ ]:
df['key'].value_counts()

## Reshaping data

### Pivoting data

People who know Excel, probably know the **Pivot** functionality:

![](img/pivot_excel.png)

The data of the table:

In [ ]:
excelample = pd.DataFrame({'Month': ["January", "January", "January", "January", 
                                  "February", "February", "February", "February", 
                                  "March", "March", "March", "March"],
                   'Category': ["Transportation", "Grocery", "Household", "Entertainment",
                                "Transportation", "Grocery", "Household", "Entertainment",
                                "Transportation", "Grocery", "Household", "Entertainment"],
                   'Amount': [74., 235., 175., 100., 115., 240., 225., 125., 90., 260., 200., 120.]})

In [ ]:
excelample

In [ ]:
excelample_pivot = excelample.pivot(index="Category", columns="Month", values="Amount")
excelample_pivot

Interested in *Grand totals*?

In [ ]:
# sum columns
excelample_pivot.sum(axis=1)

In [ ]:
# sum rows
excelample_pivot.sum(axis=0)

## Pivot is just reordering your data:

Small subsample of the titanic dataset:

In [ ]:
df = pd.DataFrame({'Fare': [7.25, 71.2833, 51.8625, 30.0708, 7.8542, 13.0],
                   'Pclass': [3, 1, 1, 2, 3, 2],
                   'Sex': ['male', 'female', 'male', 'female', 'female', 'male'],
                   'Survived': [0, 1, 0, 1, 0, 1]})

In [ ]:
df

In [ ]:
df.pivot(index='Pclass', columns='Sex', values='Fare')

In [ ]:
df.pivot(index='Pclass', columns='Sex', values='Survived')

<div class="alert alert-danger">

<b>NOTE</b>:

 <ul>
  <li>**Pivot** is purely restructering: a single value for each index/column combination is required.</li>
</ul>
</div>

# Pivot tables - aggregating while pivoting

In [ ]:
df.pivot_table(index='Sex', columns='Pclass', values='Fare')

<div class="alert alert-info">

<b>REMEMBER</b>:

 <ul>
  <li>By default, `pivot_table` takes the **mean** of all values that would end up into one cell. However, you can also specify other aggregation functions using the `aggfunc` keyword.</li>
</ul>
</div>

In [ ]:
df.pivot_table(index='Sex', columns='Pclass', 
               values='Fare', aggfunc='max')

In [ ]:
df.pivot_table(index='Sex', columns='Pclass', 
               values='Fare', aggfunc='count')

# Reshaping with `stack` and `unstack`

The docs say:

> Pivot a level of the (possibly hierarchical) column labels, returning a
DataFrame (or Series in the case of an object with a single level of
column labels) having a hierarchical index with a new inner-most level
of row labels.

Indeed... 
<img src="img/schema-stack.svg" width=50%>

Before we speak about `hierarchical index`, first check it in practice on the following dummy example:

In [ ]:
df = pd.DataFrame({'A':['one', 'one', 'two', 'two'], 
                   'B':['a', 'b', 'a', 'b'], 
                   'C':range(4)})
df

To use `stack`/`unstack`, we need the values we want to shift from rows to columns or the other way around as the index:

In [ ]:
df = df.set_index(['A', 'B']) # Indeed, you can combine two indices
df

In [ ]:
result = df['C'].unstack()
result

In [ ]:
df = result.stack().reset_index(name='C')
df

<div class="alert alert-info">

<b>REMEMBER</b>:

 <ul>
  <li>**stack**: make your data *longer* and *smaller* </li>
  <li>**unstack**: make your data *shorter* and *wider* </li>
</ul>
</div>

## To learn more on Pandas and scientific Python in general

   - https://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/tree/master/notebooks/